In [1]:
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import optuna
import os
import json
from collections import defaultdict

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split

from src.utils import get_kfold_data, convert_non_numeric_to_numeric, calculate_r2_score, calculate_metrics
from src.normalisation import Normaliser
from src.constants import *


c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv(DATA_PATH)

In [3]:
data.head()

,outcome,carat,cut,color,clarity,depth,table,price,x,y,...,a6,a7,a8,a9,a10,b6,b7,b8,b9,b10
0,-26.701232,1.14,Ideal,G,VS1,62.3,56.0,7948,6.73,6.70,...,0.168836,-0.273758,1.107832,1.247795,0.482344,0.489511,-0.321138,0.573382,0.446871,-1.990581
1,6.548093,0.38,Premium,H,VS2,60.5,59.0,898,4.69,4.66,...,-0.256549,0.315373,-0.030326,-0.114335,-1.059588,-1.761360,-1.343951,-1.002550,-0.225030,-0.446653
2,6.612562,0.50,Very Good,E,SI1,60.7,58.0,1351,5.09,5.13,...,-1.193327,-0.657307,-0.591726,-0.446856,-0.765286,-0.816544,-1.397794,-0.477130,0.810509,1.725131
3,-5.073562,0.70,Premium,D,SI1,61.2,58.0,2512,5.74,5.70,...,-1.740788,-1.778860,-0.825070,0.444932,1.173109,0.453606,-0.263440,0.246210,-0.850503,-0.412950
4,-14.436557,0.83,Ideal,G,SI2,62.4,54.0,2751,6.01,6.08,...,-0.859322,1.409268,0.861992,1.109063,-1.436722,-1.461618,0.081787,0.258087,0.851146,2.204813


Inspecting columns

In [4]:
# Find columns
all_columns = data.columns.tolist()
print(all_columns)

numeric_columns = data.select_dtypes(include=["number"]).columns.tolist()
numeric_columns.remove("outcome") # Remove the target column
print(numeric_columns)

non_numeric_columns = data.select_dtypes(exclude=["number"]).columns.tolist()
print(non_numeric_columns)

['outcome', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']
['carat', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']
['cut', 'color', 'clarity']


In [5]:
for non_numeric_column in non_numeric_columns:
    print(data[non_numeric_column].value_counts())

cut
Ideal        4040
Premium      2439
Very Good    2296
Good          925
Fair          300
Name: count, dtype: int64
color
G    2120
E    1873
F    1746
H    1506
D    1246
I     983
J     526
Name: count, dtype: int64
clarity
SI1     2408
VS2     2256
SI2     1743
VS1     1503
VVS2     951
VVS1     675
IF       318
I1       146
Name: count, dtype: int64


Converting non-numeric features to numerical features

In [6]:
data = convert_non_numeric_to_numeric(data=data)
print(data)

['G', 'E', 'F', 'H', 'D', 'I', 'J']
        outcome  carat  cut  clarity  depth  table  price     x     y     z  \
0    -26.701232   1.14    0        3   62.3   56.0   7948  6.73  6.70  4.18   
1      6.548093   0.38    1        4   60.5   59.0    898  4.69  4.66  2.83   
2      6.612562   0.50    2        5   60.7   58.0   1351  5.09  5.13  3.10   
3     -5.073562   0.70    1        5   61.2   58.0   2512  5.74  5.70  3.50   
4    -14.436557   0.83    0        6   62.4   54.0   2751  6.01  6.08  3.77   
...         ...    ...  ...      ...    ...    ...    ...   ...   ...   ...   
9995  10.718277   0.33    0        3   62.6   57.0   1002  4.42  4.40  2.76   
9996 -12.246698   1.01    4        5   69.5   55.0   4853  6.00  5.94  4.15   
9997  11.122516   0.52    2        6   57.9   61.0   1273  5.28  5.33  3.07   
9998 -24.730782   0.31    0        0   62.0   54.0    801  4.35  4.39  2.71   
9999   8.735755   0.37    2        5   59.9   59.0    649  4.68  4.70  2.81   

      ...      

Normalise data using each columns respective mean and std.

In [7]:
print(data)

        outcome  carat  cut  clarity  depth  table  price     x     y     z  \
0    -26.701232   1.14    0        3   62.3   56.0   7948  6.73  6.70  4.18   
1      6.548093   0.38    1        4   60.5   59.0    898  4.69  4.66  2.83   
2      6.612562   0.50    2        5   60.7   58.0   1351  5.09  5.13  3.10   
3     -5.073562   0.70    1        5   61.2   58.0   2512  5.74  5.70  3.50   
4    -14.436557   0.83    0        6   62.4   54.0   2751  6.01  6.08  3.77   
...         ...    ...  ...      ...    ...    ...    ...   ...   ...   ...   
9995  10.718277   0.33    0        3   62.6   57.0   1002  4.42  4.40  2.76   
9996 -12.246698   1.01    4        5   69.5   55.0   4853  6.00  5.94  4.15   
9997  11.122516   0.52    2        6   57.9   61.0   1273  5.28  5.33  3.07   
9998 -24.730782   0.31    0        0   62.0   54.0    801  4.35  4.39  2.71   
9999   8.735755   0.37    2        5   59.9   59.0    649  4.68  4.70  2.81   

      ...        b8        b9       b10  colour_G  

In [8]:
normaliser = Normaliser()
for column in numeric_columns:
    print(data[column])
    data[column] = normaliser.standardise(data[column])
    print("after", data[column])

0       1.14
1       0.38
2       0.50
3       0.70
4       0.83
        ... 
9995    0.33
9996    1.01
9997    0.52
9998    0.31
9999    0.37
Name: carat, Length: 10000, dtype: float64
after 0       0.723643
1      -0.886369
2      -0.632156
3      -0.208469
4       0.066928
          ...   
9995   -0.992290
9996    0.448246
9997   -0.589788
9998   -1.034659
9999   -0.907553
Name: carat, Length: 10000, dtype: float64
0       62.3
1       60.5
2       60.7
3       61.2
4       62.4
        ... 
9995    62.6
9996    69.5
9997    57.9
9998    62.0
9999    59.9
Name: depth, Length: 10000, dtype: float64
after 0       0.386072
1      -0.872995
2      -0.733098
3      -0.383358
4       0.456020
          ...   
9995    0.595916
9996    5.422336
9997   -2.691646
9998    0.176227
9999   -1.292683
Name: depth, Length: 10000, dtype: float64
0       56.0
1       59.0
2       58.0
3       58.0
4       54.0
        ... 
9995    57.0
9996    55.0
9997    61.0
9998    54.0
9999    59.0
Name: table, 

In [9]:
print(data)

        outcome     carat  cut  clarity     depth     table     price  \
0    -26.701232  0.723643    0        3  0.386072 -0.653020  1.024563   
1      6.548093 -0.886369    1        4 -0.872995  0.682072 -0.764609   
2      6.612562 -0.632156    2        5 -0.733098  0.237041 -0.649645   
3     -5.073562 -0.208469    1        5 -0.383358  0.237041 -0.355003   
4    -14.436557  0.066928    0        6  0.456020 -1.543082 -0.294349   
...         ...       ...  ...      ...       ...       ...       ...   
9995  10.718277 -0.992290    0        3  0.595916 -0.207990 -0.738215   
9996 -12.246698  0.448246    4        5  5.422336 -1.098051  0.239104   
9997  11.122516 -0.589788    2        6 -2.691646  1.572133 -0.669440   
9998 -24.730782 -1.034659    0        0  0.176227 -1.543082 -0.789226   
9999   8.735755 -0.907553    2        5 -1.292683  0.682072 -0.827801   

             x         y         z  ...        b8        b9       b10  \
0     0.893417  0.780367  0.923092  ...  0.593856 

Data splitting:
- Split the entire dataset into training and testing sets first.
- Use the training set to generate folds (one validation and the rest training folds) (K-Fold Cross Validation)

In [10]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=REPRODUCIBILITY_SEED)
print(f"Training set size: {len(train_data)} | Test set size: {len(test_data)}")    
print()

kfold_data = get_kfold_data(data=train_data, k=NUM_FOLDS, reproducibility_seed=REPRODUCIBILITY_SEED)

Training set size: 8000 | Test set size: 2000

Fold: 0/5
Train shape: (6400, 37) | 80.00%
Validation shape: (1600, 37) | 20.00%

Fold: 1/5
Train shape: (6400, 37) | 80.00%
Validation shape: (1600, 37) | 20.00%

Fold: 2/5
Train shape: (6400, 37) | 80.00%
Validation shape: (1600, 37) | 20.00%

Fold: 3/5
Train shape: (6400, 37) | 80.00%
Validation shape: (1600, 37) | 20.00%

Fold: 4/5
Train shape: (6400, 37) | 80.00%
Validation shape: (1600, 37) | 20.00%



Define models and hyperparameter tuning objectives for each model

In [11]:
models = {
        "linear_regression": LinearRegression,
        "lasso": Lasso,
        "ridge": Ridge,
        "xgb": xgb.XGBRegressor,
        "random_forest": RandomForestRegressor,
        "gradient_boosting": GradientBoostingRegressor,
        "ada_boost": AdaBoostRegressor,
        "lgbm": lgb.LGBMRegressor
        }

def objective(model_type, trial, x_train, y_train, x_val, y_val):
    if model_type == LinearRegression:
        parameters = {
            "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
        }
    elif model_type == Lasso:
        parameters = {
            "alpha": trial.suggest_float("alpha", 1e-3, 0.1, log=True),
            "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
            "selection": trial.suggest_categorical("selection", ["cyclic", "random"]),
            "warm_start": trial.suggest_categorical("warm_start", [True, False]),
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == Ridge:
        parameters = {
            "alpha": trial.suggest_float("alpha", 1e-3, 0.1, log=True),
            "solver": trial.suggest_categorical("solver", ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"]),
            "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
            "positive": False,
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == xgb.XGBRegressor:
        parameters = {
            "objective": "reg:squarederror",
            "eval_metric": "rmse",
            "n_estimators": 100,
            "eta": trial.suggest_float("eta", 1e-2, 0.2, log=True),
            "gamma": trial.suggest_float("gamma", 1e-8, 10, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 6),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "seed": REPRODUCIBILITY_SEED
        }
    elif model_type == RandomForestRegressor:
        parameters = {
            "n_estimators": 100,
            "criterion": trial.suggest_categorical("criterion", ["absolute_error", "squared_error"]),
            "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
            "bootstrap": True,
            "oob_score": False,
            "n_jobs": -1,
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == GradientBoostingRegressor:
        parameters = {
            "n_estimators": 100,
            "loss": trial.suggest_categorical("loss", ["absolute_error", "squared_error", "huber", "quantile"]),
            "criterion": trial.suggest_categorical("criterion", ["friedman_mse", "squared_error"]),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
            "subsample": trial.suggest_float("subsample", 0.05, 1.0),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
            "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
            "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 2, 2**10),
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == AdaBoostRegressor:
        parameters = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 100),
            "loss": trial.suggest_categorical("loss", ["linear", "square", "exponential"]),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == lgb.LGBMRegressor:
        parameters = {
                    "objective": "regression",
                    "metric": "rmse",
                    "n_estimators": 100,
                    "verbosity": -1,
                    "bagging_freq": 1,
                    "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
                    "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
                    "subsample": trial.suggest_float("subsample", 0.05, 1.0),
                    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
                    "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
                    "seed": REPRODUCIBILITY_SEED
        }

    
    model = model_type(**parameters) # Create the model
    model.fit(x_train, y_train)
    predictions = model.predict(x_val)
    metrics = calculate_metrics(targets=y_val, preds=predictions)
    rmse = metrics["rmse"]
    return rmse

In [12]:
# Train + Validate models
metrics = ["mae", "mse", "rmse", "pcc", "spearman_r", "r2_score"]
model_scores = {model_name: defaultdict(list) for model_name in models.keys()}

if os.path.exists("model_best_hyperparameters"):
    raise Exception("Directory for best hyperparameters already exists. Please delete it before running this script.")
os.makedirs("model_best_hyperparameters")

for fold in range(NUM_FOLDS):
    fold_data = kfold_data[fold]
     
    # Extract data
    train_data = fold_data["train"]
    val_data = fold_data["val"]

    train_y = train_data["outcome"]
    val_y = val_data["outcome"]
    
    train_x = train_data.drop(columns=["outcome"])
    val_x = val_data.drop(columns=["outcome"])

    # print(f"Fold {fold+1}/{NUM_FOLDS}")
    # print(f"Train data shape: {train_x.shape} | Train target shape: {train_y.shape}")
    # print(f"Val data shape: {val_x.shape} | Val target shape: {val_y.shape}")
    # print(f"Test data shape: {test_x.shape} | Test target shape: {test_y.shape}")

    # Train model
    for model_name, model in models.items():
        study = optuna.create_study(direction="minimize")
        study.optimize(lambda trial: objective(trial=trial, 
                                               model_type=model, 
                                               x_train=train_x, 
                                               y_train=train_y, 
                                               x_val=val_x, 
                                               y_val=val_y
                                               ), n_trials=N_TRIALS)
        
        # Train model with best hyperparameters
        best_fold_params = study.best_params
        model = model(**best_fold_params)
        model.fit(train_x, train_y)
        preds = model.predict(val_x)
        
        # Save the best hyperparameters for this model at this fold.
        os.makedirs(f"model_best_hyperparameters/{model_name}", exist_ok=True)
        with open(f"model_best_hyperparameters/{model_name}/fold_{fold+1}.json", "w") as f:
            json.dump(best_fold_params, f)

        # Calculate metrics
        metrics = calculate_metrics(targets=val_y, preds=preds)
        mae = metrics["mae"]
        mse = metrics["mse"]
        rmse = metrics["rmse"]
        pcc = metrics["pcc"]
        spearman_r = metrics["spearman_r"]
        r2_score = metrics["r2_score"]

        for metric in metrics:
            model_scores[model_name][metric].append(metrics[metric])

        print(f"Fold: {fold+1}/{NUM_FOLDS}")
        print(f"Model name: {model_name}")
        print(f"MAE: {mae}")
        print(f"MSE: {mse}")
        print(f"RMSE: {rmse}")
        print(f"PCC: {pcc}")
        print(f"Spearman R: {spearman_r}")
        print(f"R2 Score: {r2_score}")
        print()


[I 2025-02-17 17:19:09,875] A new study created in memory with name: no-name-17215b5f-bd45-49df-b257-7aab76cf69d2
[I 2025-02-17 17:19:09,886] Trial 0 finished with value: 10.789711542941271 and parameters: {'fit_intercept': False}. Best is trial 0 with value: 10.789711542941271.
[I 2025-02-17 17:19:09,896] Trial 1 finished with value: 10.789711542941271 and parameters: {'fit_intercept': True}. Best is trial 0 with value: 10.789711542941271.
[I 2025-02-17 17:19:09,904] Trial 2 finished with value: 10.789711542941271 and parameters: {'fit_intercept': False}. Best is trial 0 with value: 10.789711542941271.
[I 2025-02-17 17:19:09,914] Trial 3 finished with value: 10.789711542941271 and parameters: {'fit_intercept': True}. Best is trial 0 with value: 10.789711542941271.
[I 2025-02-17 17:19:09,923] Trial 4 finished with value: 10.789711542941271 and parameters: {'fit_intercept': False}. Best is trial 0 with value: 10.789711542941271.
[I 2025-02-17 17:19:09,931] Trial 5 finished with value: 1

Fold: 1/5
Model name: linear_regression
MAE: 8.70751717596474
MSE: 116.4178751798801
RMSE: 10.789711542941271
PCC: 0.5299360904553945
Spearman R: 0.5514073667606901
R2 Score: 0.2804847588737671



[I 2025-02-17 17:19:09,992] Trial 1 finished with value: 10.778365780198119 and parameters: {'alpha': 0.006636303017225869, 'fit_intercept': False, 'selection': 'cyclic', 'warm_start': True}. Best is trial 1 with value: 10.778365780198119.
[I 2025-02-17 17:19:09,999] Trial 2 finished with value: 10.772731929825774 and parameters: {'alpha': 0.02821377120279643, 'fit_intercept': True, 'selection': 'cyclic', 'warm_start': False}. Best is trial 2 with value: 10.772731929825774.
[I 2025-02-17 17:19:10,029] Trial 3 finished with value: 10.77775286056672 and parameters: {'alpha': 0.008482905903487513, 'fit_intercept': False, 'selection': 'random', 'warm_start': False}. Best is trial 2 with value: 10.772731929825774.
[I 2025-02-17 17:19:10,044] Trial 4 finished with value: 10.777015397176955 and parameters: {'alpha': 0.011898940422561783, 'fit_intercept': False, 'selection': 'cyclic', 'warm_start': False}. Best is trial 2 with value: 10.772731929825774.
[I 2025-02-17 17:19:10,111] Trial 5 fini

Fold: 1/5
Model name: lasso
MAE: 8.68876816055746
MSE: 116.04443165758475
RMSE: 10.772392104708441
PCC: 0.5319570619729997
Spearman R: 0.5540044507829887
R2 Score: 0.28279280912443683



[I 2025-02-17 17:19:10,801] Trial 0 finished with value: 10.788828698797593 and parameters: {'alpha': 0.007346256824117937, 'solver': 'saga', 'fit_intercept': True}. Best is trial 0 with value: 10.788828698797593.
[I 2025-02-17 17:19:10,808] Trial 1 finished with value: 10.789355034934347 and parameters: {'alpha': 0.042602037275108216, 'solver': 'lsqr', 'fit_intercept': False}. Best is trial 0 with value: 10.788828698797593.
[I 2025-02-17 17:19:10,812] Trial 2 finished with value: 10.789709717528646 and parameters: {'alpha': 0.001235080032974388, 'solver': 'auto', 'fit_intercept': False}. Best is trial 0 with value: 10.788828698797593.
[I 2025-02-17 17:19:11,157] Trial 3 finished with value: 10.789270796715257 and parameters: {'alpha': 0.001228709196102466, 'solver': 'sag', 'fit_intercept': True}. Best is trial 0 with value: 10.788828698797593.
[I 2025-02-17 17:19:11,163] Trial 4 finished with value: 10.789652032560891 and parameters: {'alpha': 0.006448028178453317, 'solver': 'sparse_c

...

Fold: 5/5
Model name: gradient_boosting
MAE: 7.373714989960438
MSE: 85.18869715849249
RMSE: 9.229772324304239
PCC: 0.672337582704402
Spearman R: 0.6777668038151577
R2 Score: 0.4515415880535777



[I 2025-02-17 17:22:56,932] Trial 0 finished with value: 9.322238296615584 and parameters: {'n_estimators': 93, 'loss': 'exponential', 'learning_rate': 0.06064596255063649}. Best is trial 0 with value: 9.322238296615584.
[I 2025-02-17 17:22:59,171] Trial 1 finished with value: 9.350112875782925 and parameters: {'n_estimators': 80, 'loss': 'exponential', 'learning_rate': 0.04747019313423497}. Best is trial 0 with value: 9.322238296615584.
[I 2025-02-17 17:23:01,477] Trial 2 finished with value: 9.396050539567277 and parameters: {'n_estimators': 81, 'loss': 'exponential', 'learning_rate': 0.023948839824421277}. Best is trial 0 with value: 9.322238296615584.
[I 2025-02-17 17:23:03,746] Trial 3 finished with value: 9.43185759394802 and parameters: {'n_estimators': 79, 'loss': 'linear', 'learning_rate': 0.01070341975596317}. Best is trial 0 with value: 9.322238296615584.
[I 2025-02-17 17:23:06,280] Trial 4 finished with value: 9.394737485891573 and parameters: {'n_estimators': 89, 'loss': '

Fold: 5/5
Model name: ada_boost
MAE: 7.426850861174142
MSE: 87.12109462623654
RMSE: 9.333868149177839
PCC: 0.6630597106516927
Spearman R: 0.6739915683207954
R2 Score: 0.4391005051193433



[I 2025-02-17 17:23:20,498] Trial 1 finished with value: 11.37360891473757 and parameters: {'learning_rate': 0.0023170685921187714, 'num_leaves': 916, 'subsample': 0.8060543041411522, 'colsample_bytree': 0.9333554854157214, 'min_data_in_leaf': 68}. Best is trial 0 with value: 9.070071072951906.
[I 2025-02-17 17:23:21,166] Trial 2 finished with value: 11.094886501727308 and parameters: {'learning_rate': 0.0036144388127091255, 'num_leaves': 406, 'subsample': 0.798707352614215, 'colsample_bytree': 0.7719598852723173, 'min_data_in_leaf': 2}. Best is trial 0 with value: 9.070071072951906.
[I 2025-02-17 17:23:21,646] Trial 3 finished with value: 10.367075186134937 and parameters: {'learning_rate': 0.0071287264729174345, 'num_leaves': 761, 'subsample': 0.49950168766501396, 'colsample_bytree': 0.7229458697330952, 'min_data_in_leaf': 8}. Best is trial 0 with value: 9.070071072951906.
[I 2025-02-17 17:23:21,712] Trial 4 finished with value: 9.054729004548776 and parameters: {'learning_rate': 0.0

Fold: 5/5
Model name: lgbm
MAE: 7.240123700783736
MSE: 83.0225580728234
RMSE: 9.11167153012132
PCC: 0.6838796540925558
Spearman R: 0.6891210377425929
R2 Score: 0.4654875367839687



Compute average scores and rank models by R2 score

In [13]:
for model_name, model_metrics in model_scores.items():
    for metric, scores in model_metrics.items():
        model_scores[model_name][metric] = sum(scores) / len(scores)
    model_scores[model_name] = dict(model_scores[model_name])

model_scores = dict(sorted(model_scores.items(), key=lambda x: x[1]["r2_score"], reverse=True))

In [14]:
# Print results
for i, (model_name, model_metrics) in enumerate(model_scores.items()):
    print(f"No.{i+1} Model: {model_name}")
    for metric, score in model_metrics.items():
        print(f"{metric}: {score}")
    print()

No.1 Model: xgb
mae: 7.348834685791911
mse: 85.34984311773692
rmse: 9.23768638432046
pcc: 0.6881085223299264
spearman_r: 0.693551724238955
r2_score: 0.471323852117405

No.2 Model: lgbm
mae: 7.379569888529988
mse: 86.12676660898215
rmse: 9.279862671845615
pcc: 0.6841882969072336
spearman_r: 0.6900407215002818
r2_score: 0.4665867514511738

No.3 Model: gradient_boosting
mae: 7.443432172647121
mse: 87.30004841851382
rmse: 9.34282259191409
pcc: 0.6803775050998292
spearman_r: 0.6851682215500865
r2_score: 0.4592539646620007

No.4 Model: random_forest
mae: 7.618419488992542
mse: 91.34720553814608
rmse: 9.556711006562619
pcc: 0.678904992614694
spearman_r: 0.6851569416628679
r2_score: 0.43429242738849744

No.5 Model: ada_boost
mae: 7.615960181540885
mse: 91.77427225533208
rmse: 9.57840908605571
pcc: 0.6577834727870909
spearman_r: 0.6685162617000893
r2_score: 0.431702809826693

No.6 Model: lasso
mae: 8.563903852702005
mse: 114.92431525815486
rmse: 10.71801341201545
pcc: 0.5377590357929295
spearma